In [ ]:
# !pip install pydbtools
# !pip install awswrangler
# !pip install pandas --upgrade
# !pip install time
# !pip install numpy --upgrade
# !pip install jinja2
# !pip install lovely_logger 

# import pydbtools
# import awswrangler as wr
# import model_stages.stage1_get_invest_data as stage1
# import model_stages.stage2_prepare_invests_for_merging as stage2
# import model_stages.stage3_merge_with_POA_data as stage3


# ## Config

# date_cols = ['date_received_in_investigations', 'date_allocated_to_team',
#                   'date_allocated_to_current_investigator', 'pg_sign_off_date',
#                   'closure_date',
#                   'date_received_in_opg', 'legal_approval_date_clean',
#                  'receiptdate', 'registrationdate']

# cols_to_keep = ['unique_id', 
#                 # 'client_donor_title',
#                 # 'client_donor_forename', 'client_donor_surname', 'client_donor_dob',
#                 'case_type', 'concern_type', 'status', 'sub_status',
#                 'date_received_in_opg', 'multiple_id', 'lead_case',
#                 'days_to_pg_sign_off', 'closure_date', 'case_uid', 'receiptdate', 'registrationdate',
#                 'case_status', 'poa_case_type', 'casesubtype', 'poas_involved',
#                 'order_involved', 'legal_approval_date_clean', 'invest_concluded',
#                 'year_concluded', 'case_type_agg']
# first_date = '01/01/2019'
# last_date = '31/12/2024'

# min_LPA_receiptdate = '2017-01-01'
# max_LPA_receiptdate = '2024-12-31'
# sample_size = '1000000'


# ## Read/Link Data

# # Read Investigations Data
# df_inv_data = stage1.main(date_cols, first_date, last_date, cols_to_keep)

# # Expand Investigations Data
# expanded_df = stage2.main(df_inv_data)

# # Link to LPA Data
# linked_df = stage3.main(expanded_df, 
#                         min_LPA_receiptdate, 
#                         max_LPA_receiptdate, 
#                         sample_size)

# # print(linked_df)

# linked_Investigation_LPA_data = linked_df
# linked_Investigation_LPA_data.to_csv('linked_Investigation_LPA_data.csv', index=False)

# linked_df

# inv = linked_df.loc[linked_df['investigator'].notnull()]
# inv.to_csv('linked_Investigation_LPA_inv.csv', index=False)

# inv_linked_lpa = linked_df[['uid','donor_id','lpa_reg_date','lpa_status','lpa_rec_date','poa_type','unique_id','case_no','client_donor_dob','case_type','concern_type','date_received_in_opg','status','mojap_extract_date','poa_case_type','casesubtype','poa_rec_to_invest_rec','year_concluded','link_id','uid_to_link']]
# inv_linked_lpa
# inv_linked_lpa_data = inv_linked_lpa
# inv_linked_lpa_data.to_csv('inv_linked_lpa_data.csv', index=False)

# pydbtools.read_sql_query("SELECT DISTINCT(mojap_extract_date) FROM opg_investigations_prod.investigations ORDER BY mojap_extract_date DESC")
# lpa_dashboard = pydbtools.read_sql_query("SELECT * FROM sirius_derived.opg_lpa_dashboard LIMIT 5")

How to explain the drivers and factors that caused a step reduction in the investigation cases for Lasting Power of Attorney (LPA) demands from Office of Public Guardian (OPG)? To achieve this, basically we linked Investigation and LPA dataset which resulted a data frame in Python with the following variables (columns): linked_df[['uid','donor_id','lpa_reg_date','lpa_status','lpa_rec_date','poa_type','unique_id','case_no','client_donor_dob','case_type','concern_type','date_received_in_opg','status','mojap_extract_date','poa_case_type','casesubtype','poa_rec_to_invest_rec','year_concluded','link_id','uid_to_link']].   Regarding these, can you explain what machine learning and statistical techniques is useful and how to implement them to show 1. How to investigate the age distribution (based on the dob of donor ('client_donor_dob') at the investigation ('date_received_in_opg') changes before and after pandemic that might be the reason for the step reduction in investigation of LPA cases? 2. How to show that different 'casesubtype' and 'case_type' changes influenced this step reduction in the investigation of LPA cases? 3. investigate whether: •	It suggests that the downward trend in the investigations rate for Health and Welfare cases or where investigations have included both Health and Welfare AND Property and Finance concerns have been gradual since 2016 rather than a step reduction associated with the pandemic. Having said that the rate of investigations particularly for Health and Welfare cases levelled off after the pandemic.
•	There isn’t any evidence of a gradual decline in Finance and Property cases, but instead the pattern that we have discussed before of a sustained stepped reduction following the pandemic.
•	The gradual decline in Health and Welfare and combined concerns from 2016 is interesting because it also coincides with what I believe was an operational decision at that time to remove the triage process for LPA investigations. This had the immediate effect of increasing the number of concerns accepted for investigation, which can be seen in the attached charts, followed by a gradual decline.  


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
!pip install --upgrade numpy
# !pip uninstall numpy
# !pip install numpy
import numpy as np
# print(np.__version__)
# print(np.__path__)
linked_df = pd.read_csv('inv_linked_lpa_data.csv')
linked_df

1. Investigating Age Distribution Changes Pre- and Post-Pandemic
To check whether changes in donor age at the time of investigation contributed to the reduction:

Techniques:
Descriptive Statistics & Visualization: Calculate mean, median, and IQR of donor age pre- and post-pandemic.

Kernel Density Estimation (KDE) & Histograms: Compare the age distributions before and after the pandemic.

Kolmogorov-Smirnov (KS) Test: Check if the distribution of ages significantly changed.

Causal Inference (Difference-in-Differences - DiD): Compare the mean age before and after the pandemic with a control period.

In [ ]:
!pip install --upgrade scipy matplotlib
#!pip install matplotlib
# !pip uninstall seaborn
# !pip install seaborn
!pip install --upgrade matplotlib
!pip install --upgrade seaborn
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp

# Convert to datetime
linked_df['client_donor_dob'] = pd.to_datetime(linked_df['client_donor_dob'],
                                               errors='coerce', dayfirst=True)

linked_df['date_received_in_opg'] = pd.to_datetime(linked_df['date_received_in_opg'])

# Calculate donor age at investigation
linked_df['donor_age_at_investigation'] = (linked_df['date_received_in_opg'] - linked_df['client_donor_dob']).dt.days / 365.25

# Split pre- and post-pandemic (assuming March 2020 as pandemic start)
pre_pandemic = linked_df[linked_df['date_received_in_opg'] < '2020-03-01']
post_pandemic = linked_df[linked_df['date_received_in_opg'] >= '2020-03-01']

# Plot KDE
plt.figure(figsize=(10, 5))
sns.kdeplot(pre_pandemic['donor_age_at_investigation'], label='Pre-Pandemic', shade=True)
sns.kdeplot(post_pandemic['donor_age_at_investigation'], label='Post-Pandemic', shade=True)
plt.legend()
plt.title('Age Distribution of Donors at Investigation')
plt.show()

# Perform KS test
ks_stat, p_value = ks_2samp(pre_pandemic['donor_age_at_investigation'], post_pandemic['donor_age_at_investigation'])
print(f"KS Statistic: {ks_stat}, P-Value: {p_value}")

# If the KS test p-value is low, the distributions are significantly different.

# If the post-pandemic mean is significantly lower, 
# older donors may have been investigated less.

1. Investigating Age Distribution Changes Pre- and Post-Pandemic
Refinement: Bayesian Analysis & Causal Inference
Rather than just comparing distributions, we can:

Use a Bayesian model to estimate how much the mean donor age changed.

Apply Causal Impact Analysis (Google’s CausalImpact package) to check if the change was due to the pandemic.

In [ ]:
# Suppress warnings from statsmodels
import warnings
warnings.filterwarnings("ignore")

!pip install --upgrade pymc3
#!python -m pip install --upgrade pip
#!pip uninstall pymc3
#!pip install git+https://github.com/pymc-devs/pymc3
# !git clone https://github.com/pymc-devs/pymc3
# !cd pymc3
# !pip install -r requirements.txt
# !python setup.py install
# !python setup.py develop
# !python -m pip install --upgrade pip
#!pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose
import pymc3 as pm
import arviz as az
print(f"Running on PyMC v{pm.__version__}")

In [ ]:
# A. Bayesian Estimation of Age Differences
#!pip install pymc3



# Define Bayesian Model
with pm.Model():
    mu_pre = pm.Normal("mu_pre", mu=70, sigma=10)  # Prior for Pre-Pandemic Age Mean
    mu_post = pm.Normal("mu_post", mu=70, sigma=10)  # Prior for Post-Pandemic Age Mean
    sigma = pm.HalfNormal("sigma", sigma=10)

    # Likelihood
    age_pre = pm.Normal("age_pre", mu=mu_pre, sigma=sigma, observed=pre_pandemic['donor_age_at_investigation'])
    age_post = pm.Normal("age_post", mu=mu_post, sigma=sigma, observed=post_pandemic['donor_age_at_investigation'])

    # Difference
    diff = pm.Deterministic("diff", mu_post - mu_pre)

    trace = pm.sample(2000, return_inferencedata=True)

# Plot Posterior Distribution of Age Difference
az.plot_posterior(trace, var_names=["diff"])

# This approach gives a probability distribution of the change in age, rather than a simple p-value.

# If most of the posterior distribution is negative, it means donor age at investigation decreased.

In [ ]:
# B. Causal Impact Analysis

from causalimpact import CausalImpact

# Convert to time series
age_ts = linked_df.groupby('year_received')['donor_age_at_investigation'].mean()

# Define pre/post intervention periods
pre_period = [0, len(age_ts[age_ts.index < 2020])-1]
post_period = [len(age_ts[age_ts.index < 2020]), len(age_ts)-1]

# Run Causal Impact
impact = CausalImpact(age_ts, pre_period, post_period)
impact.plot()
print(impact.summary())
print(impact.summary(output='report'))

# This quantifies the causal effect of the pandemic on donor age at investigation.

1. Investigating Age Distribution Changes Before and After the Pandemic
To investigate how the age distribution of donors at the time of investigation changed before and after the pandemic, you can use the following steps:

Techniques:
Descriptive Statistics: Calculate summary statistics (mean, median, standard deviation) for the age of donors before and after the pandemic.
Visualization: Use histograms, box plots, and density plots to visualize the age distribution.
Hypothesis Testing: Perform statistical tests (e.g., t-test, Mann-Whitney U test) to determine if there are significant differences in age distribution before and after the pandemic.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

# Assuming linked_df is your DataFrame
linked_df['age_at_investigation'] = (pd.to_datetime(linked_df['date_received_in_opg']) - pd.to_datetime(linked_df['client_donor_dob'])).dt.days / 365.25

# Split data into before and after pandemic
before_pandemic = linked_df[linked_df['date_received_in_opg'] < '2020-03-01']
after_pandemic = linked_df[linked_df['date_received_in_opg'] >= '2020-03-01']

# Descriptive statistics
print(before_pandemic['age_at_investigation'].describe())
print(after_pandemic['age_at_investigation'].describe())

# Visualization
plt.figure(figsize=(10, 6))
sns.histplot(before_pandemic['age_at_investigation'], color='blue', label='Before Pandemic', kde=True)
sns.histplot(after_pandemic['age_at_investigation'], color='red', label='After Pandemic', kde=True)
plt.legend()
plt.title('Age Distribution of Donors at Investigation')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Hypothesis testing
t_stat, p_value = ttest_ind(before_pandemic['age_at_investigation'], after_pandemic['age_at_investigation'])
print(f'T-test: t_stat={t_stat}, p_value={p_value}')

2. Analysing Changes in Case Types (‘casesubtype’ and ‘case_type’)
To assess whether shifts in case types contributed to the reduction:

Techniques:
Time Series Analysis: Visualizing case types over time.

Chi-Square Test: Checking if case distributions changed pre- and post-pandemic.

Logistic Regression: Predicting investigation likelihood based on case type.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import chi2_contingency

# Aggregate case types by year
linked_df['year_received'] = linked_df['date_received_in_opg'].dt.year
case_type_counts = linked_df.groupby(['year_received', 'case_type']).size().unstack()

# Plot trends
case_type_counts.plot(kind='line', figsize=(10,5), title="Case Type Trends Over Time")
plt.show()

# Chi-square test
pre_post_pivot = linked_df.pivot_table(index='case_type', columns=linked_df['date_received_in_opg'] >= '2020-03-01', aggfunc='size', fill_value=0)
chi2, p, dof, ex = chi2_contingency(pre_post_pivot)
print(f"Chi-Square Statistic: {chi2}, P-Value: {p}")

# Logistic Regression - Predicting Investigation Likelihood
linked_df['post_pandemic'] = linked_df['date_received_in_opg'] >= '2020-03-01'
model = smf.logit("post_pandemic ~ C(case_type) + C(casesubtype)", data=linked_df).fit()
print(model.summary())

# If chi-square p-value is low, case type proportions changed.

# Logistic regression shows which case types were more/less likely to be investigated post-pandemic.


2. Analyzing Changes in Case Types (‘casesubtype’ and ‘case_type’)
Refinement: NLP & Clustering
If there are text-based ‘concern_type’ descriptions, we can use NLP-based topic modeling to group similar concerns over time.

Clustering (K-Means, DBSCAN) can group case subtypes to reveal patterns.

In [ ]:
# A. Topic Modeling for Case Subtypes

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Convert case types into TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(linked_df['casesubtype'].astype(str))

# Apply LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(X)

# Display top words in each topic
for i, topic in enumerate(lda.components_):
    print(f"Topic {i}: {[vectorizer.get_feature_names_out()[j] for j in topic.argsort()[-5:]]}")

# Helps identify case subtype patterns that may have changed pre/post-pandemic.

In [ ]:
# B. Clustering Subtypes Over Time

from sklearn.cluster import KMeans

# Convert categorical case subtypes into numeric representations
linked_df['casesubtype_encoded'] = linked_df['casesubtype'].astype('category').cat.codes

# K-Means Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
linked_df['cluster'] = kmeans.fit_predict(linked_df[['casesubtype_encoded', 'year_received']])

# Visualize Changes in Clusters Over Time
sns.lineplot(data=linked_df, x='year_received', y='cluster', hue='casesubtype', marker='o')
plt.title('Case Subtype Clustering Over Time')
plt.show()

# If certain clusters disappear or emerge post-pandemic, they could explain the step reduction.

2. Showing Influence of 'casesubtype' and 'case_type' Changes
To show how different 'casesubtype' and 'case_type' changes influenced the step reduction in investigation cases, you can use:

Techniques:
Categorical Analysis: Analyze the frequency and distribution of different case subtypes and types before and after the pandemic.
Chi-Square Test: Perform chi-square tests to determine if there are significant differences in the distribution of case subtypes and types.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

# Frequency distribution
case_subtype_counts_before = before_pandemic['casesubtype'].value_counts()
case_subtype_counts_after = after_pandemic['casesubtype'].value_counts()

case_type_counts_before = before_pandemic['case_type'].value_counts()
case_type_counts_after = after_pandemic['case_type'].value_counts()

# Visualization
plt.figure(figsize=(12, 6))
sns.barplot(x=case_subtype_counts_before.index, y=case_subtype_counts_before.values, color='blue', label='Before Pandemic')
sns.barplot(x=case_subtype_counts_after.index, y=case_subtype_counts_after.values, color='red', label='After Pandemic')
plt.legend()
plt.title('Case Subtype Distribution Before and After Pandemic')
plt.xlabel('Case Subtype')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x=case_type_counts_before.index, y=case_type_counts_before.values, color='blue', label='Before Pandemic')
sns.barplot(x=case_type_counts_after.index, y=case_type_counts_after.values, color='red', label='After Pandemic')
plt.legend()
plt.title('Case Type Distribution Before and After Pandemic')
plt.xlabel('Case Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

# Chi-square test
contingency_table_subtype = pd.crosstab(linked_df['casesubtype'], linked_df['date_received_in_opg'] >= '2020-03-01')
chi2_stat_subtype, p_val_subtype, dof_subtype, ex_subtype = chi2_contingency(contingency_table_subtype)
print(f'Chi-square test for case subtype: chi2_stat={chi2_stat_subtype}, p_value={p_val_subtype}')

contingency_table_type = pd.crosstab(linked_df['case_type'], linked_df['date_received_in_opg'] >= '2020-03-01')
chi2_stat_type, p_val_type, dof_type, ex_type = chi2_contingency(contingency_table_type)
print(f'Chi-square test for case type: chi2_stat={chi2_stat_type}, p_value={p_val_type}')

3. Investigating Gradual vs. Step Decline in Investigation Cases (2016 Onwards)
To determine if the decline was gradual (since 2016) or a sharp step drop post-pandemic:

Techniques:
Time Series Decomposition: Breaking down long-term trends and seasonality.

Change Point Detection: Identifying structural breaks in investigation rates.

Interrupted Time Series Analysis (ITSA): Evaluating the impact of pandemic on investigation trends.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import ruptures as rpt

# Aggregate investigation counts by year
investigations_by_year = linked_df.groupby('year_received').size()

# Time Series Decomposition
decomposed = seasonal_decompose(investigations_by_year, model='additive', period=1)
decomposed.plot()
plt.show()

# Change Point Detection
algo = rpt.Pelt(model="rbf").fit(investigations_by_year.values.reshape(-1, 1))
breakpoints = algo.predict(pen=10)
print("Change Points Detected at Years:", [list(investigations_by_year.index)[bp] for bp in breakpoints[:-1]])

# ITSA - Comparing Pre/Post Pandemic Trends
linked_df['post_pandemic'] = (linked_df['year_received'] >= 2020).astype(int)
model = smf.ols("casesubtype ~ year_received + post_pandemic", data=linked_df).fit()
print(model.summary())

# Change point detection helps confirm whether the drop was gradual or sudden.

# ITSA helps quantify the impact of the pandemic on investigation trends.

Findings to Look for:
Age Analysis: If older donors were investigated less post-pandemic, it may explain some of the decline.

Case Type Shifts: A significant drop in certain cases (e.g., Health & Welfare) might suggest policy changes.

Trend Analysis: If Finance & Property cases show a step decline while Health & Welfare cases decline gradually, it supports the operational decision hypothesis.

3. Investigating Gradual vs. Step Decline in Investigation Cases (2016 Onwards)
Refinement: Structural Breaks & Granger Causality
To determine if the step reduction aligns with operational decisions, we can:

Detect breakpoints in investigation rates using Bayesian Change Point Detection.

Apply Granger Causality to test whether operational changes caused case volume reductions.

In [ ]:
# A. Bayesian Change Point Detection

import pymc3 as pm

# Convert investigation counts to numpy array
y = investigations_by_year.values

# Define Model
with pm.Model():
    tau = pm.DiscreteUniform("tau", lower=0, upper=len(y)-1)
    mu1 = pm.Normal("mu1", mu=np.mean(y[:len(y)//2]), sigma=np.std(y))
    mu2 = pm.Normal("mu2", mu=np.mean(y[len(y)//2:]), sigma=np.std(y))
    
    # Likelihood
    idx = np.arange(len(y))
    mu = pm.math.switch(tau > idx, mu1, mu2)
    obs = pm.Normal("obs", mu=mu, sigma=np.std(y), observed=y)

    trace = pm.sample(2000, return_inferencedata=True)

# Plot Posterior Distribution of Breakpoint
az.plot_posterior(trace, var_names=["tau"])

# If the posterior distribution of tau (change point) aligns with 2016 or the pandemic, it suggests a structural change.

In [ ]:
# B. Granger Causality for Triage Removal vs. Investigation Decline

from statsmodels.tsa.stattools import grangercausalitytests

# Creating a DataFrame with lagged operational decisions
df_granger = linked_df[['year_received', 'case_type']].pivot_table(index='year_received', columns='case_type', aggfunc='size', fill_value=0)
df_granger['triage_removed'] = (df_granger.index >= 2016).astype(int)

# Running Granger Causality Test
grangercausalitytests(df_granger[['case_type', 'triage_removed']], maxlag=5)

# If triage removal Granger-causes the decline in investigations, we have strong causal evidence.

3. Investigating Trends in Investigation Rates
To investigate trends in investigation rates for different case types and subtypes, you can use:

Techniques:
Time Series Analysis: Analyze the trends over time using line plots and statistical tests.
Regression Analysis: Use regression models to identify trends and changes over time.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

# Group by year and case type/subtype
yearly_counts = linked_df.groupby(['year_concluded', 'case_type']).size().unstack().fillna(0)
yearly_counts_subtype = linked_df.groupby(['year_concluded', 'casesubtype']).size().unstack().fillna(0)

# Visualization
plt.figure(figsize=(12, 6))
yearly_counts.plot(kind='line', marker='o')
plt.title('Yearly Investigation Counts by Case Type')
plt.xlabel('Year')
plt.ylabel('Number of Investigations')
plt.legend(title='Case Type')
plt.show()

plt.figure(figsize=(12, 6))
yearly_counts_subtype.plot(kind='line', marker='o')
plt.title('Yearly Investigation Counts by Case Subtype')
plt.xlabel('Year')
plt.ylabel('Number of Investigations')
plt.legend(title='Case Subtype')
plt.show()

# Time series decomposition
for case_type in yearly_counts.columns:
    result = seasonal_decompose(yearly_counts[case_type], model='additive', period=1)
    result.plot()
    plt.title(f'Time Series Decomposition for {case_type}')
    plt.show()

# Regression analysis

X = yearly_counts.index.values.reshape(-1, 1)
for case_type in yearly_counts.columns:
    y = yearly_counts[case_type].values
    model = sm.OLS(y, sm.add_constant(X)).fit()
    print(f'Regression analysis for {case_type}:')
    print(model.summary())

Next Steps
Deep Dive into Outliers

If we find an anomaly in 2016 or 2020, investigate if it aligns with internal reports or policy changes.

Interactive Dashboards

Use Plotly or Dash to create an interactive visualization for stakeholders.

Predictive Modeling for Future Investigations

Use XGBoost or Random Forest to predict the number of investigations based on past trends.

A dashboard will help OPG stakeholders visualize the trends, analyze case distributions, and interactively explore insights. I'll create a Plotly Dash application that includes:

Age Distribution Analysis – A histogram/KDE plot comparing pre- and post-pandemic donor ages.

Case Type Trends Over Time – A time-series line chart showing case subtypes from 2016 onward.

Investigation Volume & Structural Changes – A bar chart with annotations marking key events (e.g., triage removal in 2016, pandemic in 2020).

Case Type Breakdown (Pie Chart) – To compare case distributions over different periods.

Changepoint Detection & Forecasting – Structural break analysis visualized dynamically.

This Plotly Dash app provides an interactive dashboard for OPG stakeholders to explore:

Donor Age Distribution before and after the pandemic.

Case Type Trends Over Time (line chart).

Investigation Volume Changes with key events like triage removal in 2016.

Case Type Breakdown (pie chart).

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Load and preprocess data
linked_df['date_received_in_opg'] = pd.to_datetime(linked_df['date_received_in_opg'])
linked_df['year_received'] = linked_df['date_received_in_opg'].dt.year
linked_df['donor_age'] = (linked_df['date_received_in_opg'] - pd.to_datetime(linked_df['client_donor_dob'])).dt.days / 365.25

# Aggregate data
case_type_trend = linked_df.groupby(['year_received', 'case_type']).size().reset_index(name='count')

# Dash App
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("OPG Investigation Dashboard"),
    
    # Age Distribution Comparison
    dcc.Graph(
        figure=px.histogram(linked_df, x='donor_age', color=(linked_df['year_received'] >= 2020),
                            nbins=50, title="Donor Age Distribution: Pre vs. Post Pandemic")
    ),
    
    # Case Type Trends
    dcc.Graph(
        figure=px.line(case_type_trend, x='year_received', y='count', color='case_type',
                       title="Trends in Case Types Over Time")
    ),
    
    # Investigation Volume with Key Events
    dcc.Graph(
        figure=px.bar(linked_df.groupby('year_received').size().reset_index(name='count'), x='year_received', y='count',
                      title="Annual Investigation Volume").add_vline(x=2016, line_dash='dash', annotation_text='Triage Removed')
    ),
    
    # Case Type Breakdown (Pie Chart)
    dcc.Graph(
        figure=px.pie(linked_df, names='case_type', title="Case Type Distribution")
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


additional features, such as changepoint detection visualizations or predictive modeling? 